# Photogrammetry

## Step 0: some coordinate transformations

In [3]:
import numpy as np
from dataclasses import dataclass
from typing import Optional

@dataclass
class Point2D:
    u: float
    v: float
    
    def to_array(self) -> np.ndarray:
        return np.array([self.u, self.v])
    
    @classmethod
    def from_array(cls, arr: np.ndarray) -> 'Point2D':
        return cls(float(arr[0]), float(arr[1]))
    
    @classmethod
    def from_point3D(cls, p: Point3D) -> 'Point2D':
        return cls(p.x/p.z, p.y/p.z)

@dataclass
class Point3D:
    x: float
    y: float
    z: float
    
    def to_array(self) -> np.ndarray:
        return np.array([self.x, self.y, self.z])
    
    def to_homogeneous(self) -> np.ndarray:
        return np.array([self.x, self.y, self.z, 1.0])
    
    @classmethod
    def from_array(cls, arr: np.ndarray) -> 'Point3D':
        return cls(float(arr[0]), float(arr[1]), float(arr[2]))

@dataclass
class Line:
    # Point on line
    x: float
    y: float
    z: float
    # Direction vector (should be normalized)
    vx: float
    vy: float
    vz: float
    
    def to_arrays(self) -> tuple[np.ndarray, np.ndarray]:
        return (
            np.array([self.x, self.y, self.z]),
            np.array([self.vx, self.vy, self.vz])
        )
    
    @classmethod
    def from_arrays(cls, point: np.ndarray, direction: np.ndarray) -> 'Line':
        # Normalize direction vector
        direction = direction / np.linalg.norm(direction)
        return cls(
            float(point[0]), float(point[1]), float(point[2]),
            float(direction[0]), float(direction[1]), float(direction[2])
        )

@dataclass
class EarthCoord:
    longitude: float  # degrees
    latitude: float   # degrees
    altitude: float   # meters
    
    def to_array(self) -> np.ndarray:
        return np.array([self.longitude, self.latitude, self.altitude])
    
    @classmethod
    def from_array(cls, arr: np.ndarray) -> 'EarthCoord':
        return cls(float(arr[0]), float(arr[1]), float(arr[2]))

class Transform3D:
    def __init__(self, R: np.ndarray, t: np.ndarray):
        """
        rotation matrix R (3x3) and translation vector t (3,)
        """
        self.R = R.copy()
        self.t = t.copy()
        self.R_inv = self.R.transpose()
        self.t_inv = -self.R_inv @ t
        
    def forward_matrix(self) -> np.ndarray:
        mat = np.eye(4)
        mat[:3, :3] = self.R
        mat[:3, 3] = self.t
        return mat
    
    def backward_matrix(self) -> np.ndarray:
        mat = np.eye(4)
        mat[:3, :3] = self.R_inv
        mat[:3, 3] = self.t_inv
        return mat
    
    def forward_transform_point(self, point: Point3D) -> Point3D:
        p = point.to_array()
        p_new = self.R @ p + self.t
        return Point3D.from_array(p_new)
    
    def backward_transform_point(self, point: Point3D) -> Point3D:
        p = point.to_array()
        p_new = self.R_inv @ (p - self.t)
        return Point3D.from_array(p_new)
    
    def forward_transform_line(self, line: Line) -> Line:
        p, v = line.to_arrays()
        p_new = self.R @ p + self.t
        v_new = self.R @ v
        return Line.from_arrays(p_new, v_new)
    
    def backward_transform_line(self, line: Line) -> Line:
        p, v = line.to_arrays()
        p_new = self.R_inv @ (p - self.t)
        v_new = self.R_inv @ v
        return Line.from_arrays(p_new, v_new)

class TransformCamera:
    def __init__(self, K: np.ndarray, R: np.ndarray, t: np.ndarray):
        """
        K: 3x3 camera intrinsic matrix
        R: 3x3 camera rotation matrix
        t: 3, camera translation vector
        """
        self.K = K.copy()
        self.K_inv = np.linalg.inv(K)
        self.transform = Transform3D(R, t)
    
    def forward_projection(self, point: Point3D) -> Point2D:
        p_cam = self.transform.forward_transform_point(point)
        assert p_cam.z > 0, "The point should be in front of the camera."
        uv = self.K @ p_cam.to_array()
        return Point2D.from_point3D(uv)
    
    def backward_projection(self, point: Point2D) -> Line:
        # Get normalized direction in camera coordinates
        uv_homo = np.array([point.u, point.v, 1.0])
        direction = self.K_inv @ uv_homo
        
        # Ray starts at camera center ((0, 0, 0) in camera coordinates)
        camera_ray = Line(0.0, 0.0, 0.0, direction[0], direction[1], direction[2])
        return self.transform.backward_transform_line(camera_ray)

In [20]:
theta = 0.3
trans = Transform3D(np.array([[np.cos(theta), np.sin(theta), 0],
                              [-np.sin(theta), np.cos(theta), 0],
                              [0, 0, 1]]), 
                    np.array([1, 0, 0]))
point = Point3D(1, 2, 3)
trans.backward_transform_point(trans.forward_transform_point(point))

Point3D(x=0.9999999999999998, y=1.9999999999999998, z=3.0)

## Step 1: Camera calibration

`camera_resection: [(Point3D, Point2D)] -> TransformCamera`

In [21]:
def camera_resection(matched_points: [(Point3D, Point2D)]) -> TransformCamera:
    # Call cv2.solvePnP
    return 0



3